In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
PATH = "dataset/indexData.csv"
dataset = pd.read_csv(PATH)

In [3]:
EXCHANGE = "N100"
dataset = dataset[dataset["Index"] == EXCHANGE]
dataset = dataset.fillna(method='ffill')

price = np.array(pd.to_numeric(dataset["Close"]))

In [4]:
price.shape

(5507,)

In [5]:
TIME_STEPS = 20

train_input = np.array([])
train_output = np.array([])

for index in range(0, price.size - TIME_STEPS - 1):
    train_input = np.append(
        train_input,
        np.array(price[index: index + TIME_STEPS]),
    )
    train_output = np.append(
        train_output,
        np.array(price[index + TIME_STEPS]),
    )

train_input = train_input.reshape(int(train_input.size / TIME_STEPS), TIME_STEPS, 1)
train_output = train_output.reshape(train_output.size, 1)

In [6]:
train_input.shape

(5486, 20, 1)

In [7]:
train_output[-1]

array([1258.579956])

In [8]:
price[-2]

1258.579956

In [9]:
model = tf.keras.models.load_model("model/mse 258.h5")

In [10]:
model.evaluate(
    train_input,
    train_output
)

172/172 [==============================] - 4s 12ms/step - loss: 258.3438


258.34375

In [12]:
%%timeit

PREDICTIONS_NUMBER = 20

prediction_input = train_input.copy()
prediction_output = np.array([])

for _ in range(PREDICTIONS_NUMBER):
    prediction = model.predict(
        prediction_input[-1].reshape(1, 20, 1),
        verbose=False
    )
    prediction_output = np.append(
        prediction_output,
        prediction
    )

    new_time_series = prediction_input[-1][1:]
    new_time_series = np.append(new_time_series, [prediction])

    shape = prediction_input.shape

    prediction_input = np.append(
        prediction_input,
        [new_time_series]
    )

    prediction_input = prediction_input.reshape(
        int(prediction_input.shape[0] / shape[1]),
        shape[1],
        shape[2]
    )

prediction_output = prediction_output.reshape(
    PREDICTIONS_NUMBER,
    1
)

prediction_output

851 ms ± 25.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
